# Bidirection LSTM - IMDB sentiment classification

see **https://github.com/keras-team/keras/blob/master/examples/imdb_bidirectional_lstm.py**

In [ ]:
KERAS_MODEL_FILEPATH = '../../demos/data/imdb_bidirectional_lstm/imdb_bidirectional_lstm.h5'

In [ ]:
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Input, Bidirectional
from keras.datasets import imdb
from keras.callbacks import EarlyStopping, ModelCheckpoint

import json

Using TensorFlow backend.


In [ ]:
max_features = 20000
maxlen = 200  # cut texts after this number of words (among top max_features most common words)

print('Loading data...')
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)
print(len(X_train), 'train sequences')
print(len(X_test), 'test sequences')

print("Pad sequences (samples x time)")
X_train = sequence.pad_sequences(X_train, maxlen=maxlen)
X_test = sequence.pad_sequences(X_test, maxlen=maxlen)
print('X_train shape:', X_train.shape)
print('X_test shape:', X_test.shape)
y_train = np.array(y_train)
y_test = np.array(y_test)

Loading data...
17465344/17464789 [==============================] - 2s 0us/step
25000 train sequences
25000 test sequences
Pad sequences (samples x time)
X_train shape: (25000, 200)
X_test shape: (25000, 200)


In [ ]:
model = Sequential()
model.add(Embedding(max_features, 64, input_length=maxlen))
model.add(Bidirectional(LSTM(32)))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
model.compile('adam', 'binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Model saving callback
checkpointer = ModelCheckpoint(filepath=KERAS_MODEL_FILEPATH, monitor='val_acc', verbose=1, save_best_only=True)

# Early stopping
early_stopping = EarlyStopping(monitor='val_acc', verbose=1, patience=2)

# train
batch_size = 128
epochs = 10
model.fit(X_train, y_train, 
          validation_data=[X_test, y_test],
          batch_size=batch_size, epochs=epochs, verbose=2,
          callbacks=[checkpointer, early_stopping])

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/indexed_slices.py:434: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 25000 samples, validate on 25000 samples
Epoch 1/10
 - 80s - loss: 0.4698 - accuracy: 0.7723 - val_loss: 0.3175 - val_accuracy: 0.8683
Epoch 2/10


/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/keras/callbacks/callbacks.py:846: RuntimeWarning: Early stopping conditioned on metric `val_acc` which is not available. Available metrics are: val_loss,val_accuracy,loss,accuracy
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


 - 78s - loss: 0.2313 - accuracy: 0.9166 - val_loss: 0.3111 - val_accuracy: 0.8676
Epoch 3/10
 - 77s - loss: 0.1567 - accuracy: 0.9495 - val_loss: 0.3727 - val_accuracy: 0.8626
Epoch 4/10
 - 78s - loss: 0.1030 - accuracy: 0.9677 - val_loss: 0.4567 - val_accuracy: 0.8575
Epoch 5/10
 - 78s - loss: 0.0802 - accuracy: 0.9758 - val_loss: 0.4204 - val_accuracy: 0.8608
Epoch 6/10
 - 77s - loss: 0.0562 - accuracy: 0.9838 - val_loss: 0.5201 - val_accuracy: 0.8518
Epoch 7/10
 - 78s - loss: 0.0558 - accuracy: 0.9837 - val_loss: 0.5900 - val_accuracy: 0.8512
Epoch 8/10
 - 77s - loss: 0.0614 - accuracy: 0.9802 - val_loss: 0.6170 - val_accuracy: 0.8478
Epoch 9/10


**sample data**

In [ ]:
word_index = imdb.get_word_index()

In [ ]:
word_dict = {idx: word for word, idx in word_index.items()}

In [ ]:
sample = []
for idx in X_train[0]:
    if idx >= 3:
        sample.append(word_dict[idx-3])
    elif idx == 2:
        sample.append('-')
' '.join(sample)

In [ ]:
with open('../../demos/data/imdb_bidirectional_lstm/imdb_dataset_word_index_top20000.json', 'w') as f:
    f.write(json.dumps({word: idx for word, idx in word_index.items() if idx < max_features}))

In [ ]:
with open('../../demos/data/imdb_bidirectional_lstm/imdb_dataset_word_dict_top20000.json', 'w') as f:
    f.write(json.dumps({idx: word for word, idx in word_index.items() if idx < max_features}))

In [ ]:
sample_test_data = []
for i in np.random.choice(range(X_test.shape[0]), size=1000, replace=False):
    sample_test_data.append({'values': X_test[i].tolist(), 'label': y_test[i].tolist()})
    
with open('../../demos/data/imdb_bidirectional_lstm/imdb_dataset_test.json', 'w') as f:
    f.write(json.dumps(sample_test_data))